In [ ]:
import pandas as pd
import pandas as pd
import json
from zhipuai import ZhipuAI
import re
import requests
fold = "./新建文件夹/"
api_key="f689043de7886e8c604802325fa16392.9m8y9PP2XBKP9ZJy"
access_token="0099fe8c9593475d96195107b7acf7bd"

In [ ]:

# 读取Excel文件，这里假设文件名为 'your_file.xlsx'，你可以替换成实际的文件名及路径
excel_file = pd.ExcelFile('数据字典.xlsx')

# 获取第一个sheet页的内容
first_sheet_data = excel_file.parse(0)  # 索引0代表第一个sheet页，也可以传入sheet页的名称
a = excel_file.parse(0).to_json(orient="records", force_ascii=False)

# 获取第二个sheet页的内容
second_sheet_data = excel_file.parse(1)  # 索引1代表第二个sheet页，同样也可传入sheet页的名称
b = excel_file.parse(0).to_json(orient="records", force_ascii=False)

In [ ]:

d = []
question = json.load(open("question.json", 'r', encoding='utf-8'))
for question_one in question:
    chat_message = ""
    for question_team in question_one['team']:
        chat_message += question_team['question']
        client = ZhipuAI(api_key=api_key) 
        response = client.chat.completions.create(
            model="glm-4-air",  
            messages=[
                {"role": "user", "content": """已知以下数据库 """+a+""" 确定解决问题 """+chat_message+""" 所需要的库名与表名""" },
                {"role": "user", "content": "按照json格式返回 结构与数据库结构一致 只返回json其他的什么都不要逼逼"}
            ],
        )

        print(response.choices[0].message)
        d.append({"question": question_team['question'],'历史对话':chat_message, '相关数据库字段': response.choices[0].message.content})
        json.dump(d, open("从问题到数据库表结构.json", 'w', encoding="utf-8"), ensure_ascii=False)

In [ ]:

# 读取Excel文件，这里假设文件名为 '数据字典.xlsx'，你可以替换成实际的文件名及路径
excel_file = pd.ExcelFile('数据字典.xlsx')

# 获取第一个sheet页的内容
first_sheet_data = excel_file.parse(0)  # 索引0代表第一个sheet页，也可以传入sheet页的名称
a = excel_file.parse(0).to_json(orient="records", force_ascii=False)

# 获取第二个sheet页的内容
second_sheet_data = excel_file.parse(1)  # 索引1代表第二个sheet页，同样也可传入sheet页的名称
b = excel_file.parse(1).to_json(orient="records", force_ascii=False)
import json

b_data = json.loads(b)
从问题到数据库结构 = json.load(open("从问题到数据库结构.json",'r',encoding="utf-8"))
for i in 从问题到数据库结构:
    i["相关数据字段"] = []
    for j in b_data:
        try:
            if j['table_name'] in i['相关数据库字段']:
                i["相关数据字段"].append(j)
        except:
            continue
json.dump(从问题到数据库结构,open("根据库表结构匹配字段.json",'w'),ensure_ascii=False)

In [ ]:

d = []
question = json.load(open("根据库表结构匹配字段.json", 'r', encoding='utf-8'))
for question_one in 从问题到数据库结构:
        client = ZhipuAI(api_key=api_key)  # 请填写您自己的APIKey
        response = client.chat.completions.create(
            model="glm-4-air",  # 请填写您要调用的模型名称
            messages=[
                {"role": "user", "content": """解决问题：""" + str(question_one['历史对话'])+"需要以下哪些数据库字段支持："+str(question_one['相关数据库字段'])},
                {"role": "user", "content": "按照数据库字段的json格式返回"}
            ],
        )

        print(response.choices[0].message)
        question_one['适配字段'] = response.choices[0].message.content
        d.append(question_one)
        json.dump(d, open("根据问题+表结构+字段找到适配字段.json", 'w', encoding="utf-8"), ensure_ascii=False)

In [ ]:

d = []
question = json.load(open("根据问题+表结构+字段找到适配字段.json", 'r', encoding='utf-8'))
for question_one in question:
        question_one.pop('相关数据库字段')
        client = ZhipuAI(api_key=api_key)  # 请填写您自己的APIKey
        response = client.chat.completions.create(
            model="glm-4-air",  # 请填写您要调用的模型名称
            messages=[
                {"role": "user", "content": """示例sql ：SELECT * FROM constantdb.secumain LIMIT 10 
                要求 库名.表名 按照需求来进行sql生成 不是让你改写这个sql 然你根据问题和增强信息来生成一段可以解决问题的sql
                已知：""" + str(question_one)},
                {"role": "user", "content": "按照sql格式返回"}
            ],
        )

        print(response.choices[0].message.content)
        question_one['sql'] = response.choices[0].message.content
        d.append(question_one)
        json.dump(d, open("根据问题+表结构+字段生成sql.json", 'w', encoding="utf-8"), ensure_ascii=False)


In [ ]:
data = json.load(open(fold+"根据问题+表结构+字段生成sql.json", 'r', encoding='utf-8'))

In [ ]:
pattern = r'(?s)\```sql\n(.*?)\n\```'
# 假设 data 是你的数据源
d = []
for item in data:
    if isinstance(item, dict) and "sql" in item:
        sql_statement = item["sql"]
    match = re.search(pattern, sql_statement)
    if match:
        extracted_sql = match.group(1)
        d.append(extracted_sql)
        with open(fold+'sql.json', 'w', encoding='utf-8') as f:
            json.dump(d, f, ensure_ascii=False, indent=4)

In [ ]:
data = json.load(open(fold+"sql.json", 'r', encoding='utf-8'))

In [ ]:
d = []
for item in data:
    url = "https://comm.chatglm.cn/finglm2/api/query"
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer "+access_token
    }
    data = {
        "sql": item,
        "limit": 10
    }
    response = requests.post(url, headers=headers, json=data)
    result = response.json()

    # 将结果追加到sql_data_one列表中
    d.append(result)
    # 每次追加后立即保存到JSON文件
    with open(fold+'result.json', 'w', encoding='utf-8') as f:
        json.dump(d, f, ensure_ascii=False, indent=4)

In [ ]:
def llm_chat(api_key,content):
        client = ZhipuAI(api_key=api_key)  
        response = client.chat.completions.create(
            model="glm-4-air", 
            messages=[
                {"role": "user", "content": content},
            ],
        )
        return response.choices[0].message.content
    
def laod_json(file):
    return json.load(open(file, 'r', encoding='utf-8'))

def write_json(file,d):
    with open(file, 'w', encoding='utf-8') as f:
        json.dump(d, f, ensure_ascii=False, indent=4)

In [ ]:
llm_chat(api_key,"你好")

In [136]:
result = json.load(open(fold+"result.json", 'r', encoding='utf-8'))
example = json.load(open("submit_example.json", 'r', encoding='utf-8'))
question_all = json.load(open(fold+'question_all.json', 'r', encoding='utf-8'))

In [137]:
d = []
i = 0
for item in example:
    for team in item["team"]:
        q = team["question"]
        r = result[i]
        if "data" in r:
            prompt = "请你根据查询结果回答问题，问题如下:\n"+q+"\n查询结果如下:\n"+str(r["data"])+"\n"
            answer = llm_chat(api_key,prompt)
            team["answer"] = answer
            with open(fold+'answer2.json', 'w', encoding='utf-8') as f:
                json.dump(example, f, ensure_ascii=False, indent=4)
        i=i+1


In [ ]:
from tqdm import tqdm
d = []
for i in tqdm(range(len(question_all)), desc="Processing", unit="item"):
    r = result[i]
    if "data" in r:
        q = question_all[i]
        prompt = "请你根据查询结果回答问题，问题如下:\n"+q+"\n查询结果如下:\n"+str(r["data"])+"\n"
        answer = llm_chat(api_key,prompt)
        d.append(answer)
        with open(fold+'sql2answer.json', 'w', encoding='utf-8') as f:
            json.dump(d, f, ensure_ascii=False, indent=4)

In [ ]:
example = json.load(open("submit_example.json", 'r', encoding='utf-8'))


In [ ]:
i = 0
for item in example:
    for team in item["team"]:
        q = team["question"]